In [1]:
import os

try:
    snakemake
except NameError:
    from snakemk_util import load_rule_args
    
    os.chdir('../..')
    
    snakemake = load_rule_args(
        snakefile = os.getcwd() + "/workflow/Snakefile",
        rule_name = 'bam_read_count_stats',
        root = '..'
    )

In [10]:
from pathlib import Path
import pandas as pd


counts = dict()

for i in snakemake.input['counts']:
    counts[Path(i).stem] = int(open(i).read())

df_counts = pd.Series(counts)

In [3]:
df

ENCFF292UIE    3201257
ENCFF738RAA    2753910
ENCFF319JFG     873315
ENCFF901XCR    1146669
ENCFF319FBW     801570
                ...   
ENCFF623IBV    1014576
ENCFF243PFI     612514
ENCFF171AQO    1288131
ENCFF903XBV    2613456
ENCFF349CHO    2492711
Length: 138, dtype: int64

In [12]:
df_mapping = pd.read_csv(snakemake.input['mapping'], sep='\t', header=None).set_index(0)

df_read_annot_counts = pd.read_csv(snakemake.input['read_annot_counts'], sep='\t')


In [13]:
df_mapping

,1
0,
ENCFF118JEI,mcf7_1_1
ENCFF322UJU,k562_1_1
ENCFF660UTF,heart_right_ventricle_2_1
ENCFF219UJG,gm12878_3_1
ENCFF018PZX,heart_right_ventricle_3_1
...,...
ENCFF927MKK,pgp1_endo_1_2
ENCFF344KQF,left_cardiac_atrium_1_1
ENCFF319JFG,h9_neural_crest_1_1


In [14]:
df_read_annot_counts

,dataset,counts
0,a673_1_1,1330194
1,a673_1_2,1707207
2,adrenal_gland_1_1,765655
3,adrenal_gland_2_1,1579294
4,adrenal_gland_3_1,577077
...,...,...
133,right_ventricle_myocardium_superior_1_1,1560166
134,upper_lobe_of_right_lung_1_1,1445367
135,wtc11_1_1,2332639
136,wtc11_1_2,1358180


In [21]:
df = df_mapping.join(df_counts.to_frame())
df = df.set_index(1).join(df_read_annot_counts.set_index('dataset'))

In [25]:
df['dif'] = df[0] - df['counts']

In [29]:
df.sort_values('dif', ascending=False).head(50)

,0,counts,dif
1,,,
cardiac_septum_1_1,1769114,1463725,305389
lower_lobe_of_right_lung_1_1,1803044,1528088,274956
psoas_muscle_1_1,1756513,1508710,247803
right_cardiac_atrium_4_1,1737745,1492063,245682
wtc11_1_2,1603742,1358180,245562
mucosa_of_descending_colon_2_1,1288131,1105691,182440
upper_lobe_of_right_lung_1_1,1620966,1445367,175599
wtc11_1_1,2505856,2332639,173217
aorta_2_1,1107999,945858,162141


In [ ]:


import pdb
pdb.set_trace()


df.to_csv(snakemake.output['table'], sep='\t')